In [12]:
from gensim.models import word2vec
import nltk
import re
from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;
import pandas as pd;
import numpy as np;
from itertools import cycle;
from wordcloud import WordCloud, ImageColorGenerator
import wikipedia
import sys
import requests
import bs4
import urllib.request
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from os import path
import wikipedia
import sys
import requests
import bs4
import urllib.request
import re
from nltk.corpus import stopwords 
#from nltk.tokenize import word_tokenize
from nltk import tokenize
from os import path

#level 0 data
res= requests.get('https://en.wikipedia.org/wiki/'+'Sinhagad')
res.raise_for_status()

wiki=bs4.BeautifulSoup(res.text,"html.parser")  
level0data=' '
for i in wiki.select('p'):
    print(i.getText())
    level0data=level0data + i.getText()

#To convert in list of list
sentences=[]
sentences=tokenize.sent_tokenize(level0data)
print(sentences)
   
words=[]
for sentence in sentences:
        sub = sentence.split(" ")
        words.append(sub) 
print(words)


#Remove stop words
stopwords = set(stopwords.words('english'))
words_reviews=[]
for review in words:
    wr=[]
    for word in review:
        if word not in stopwords:
            wr.append(word)
        words_reviews.append(wr)      
print("Without stop words", words_reviews)

#Model formation
num_features = 100;    # Dimensionality of the hidden layer representation
min_word_count = 40;   # Minimum word count to keep a word in the vocabulary
num_workers =1;       # Number of threads to run in parallel set to total number of cpus.
context = 5          # Context window size (on each side) 
#model = word2vec.Word2Vec(words_reviews, workers = 1, size = 100, min_count = 10, window = 5, sg = 0)
model = word2vec.Word2Vec(words_reviews, workers=num_workers,size=num_features, min_count = min_word_count,window = context);

model_name = "model_sinhagad";
model.save(model_name);
Z = model.wv.syn0;#Raw vector storage
print(Z[0].shape)
print(Z[0])

#kmeans clustering
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    return kmeans_clustering.cluster_centers_, idx;

def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
    #Closest points for each Cluster center is used to query the closest k points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
    #Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
    #A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    print (df)
    return df;


centers, clusters = clustering_on_wordvecs(Z, 5);
#index2word= list of word-vectors has moved to the wv property of the model
top_words = get_top_words(model.wv.index2word, 50, centers, Z);
print("Center count",centers.shape)
print("Centers, ",centers)
print("Clusters count",clusters.shape)
print("Clusters",clusters)






Sinhagad is a hill fortress located at around 35 km southwest of the city of Pune, India. Some of the information available at this fort suggests that the fort could have been built 2000 years ago. The caves and the carvings in the Kaundinyeshwar temple stand as proofs for the same.

Previously known as Kondhana, the fort had been the site of many battles, most notably the Battle of Sinhagad in 1670. Perched on an isolated cliff of the Bhuleswar range in the Sahyadri Mountains, the fort is situated on a hill about 760 metres above ground and 1,312 metres above mean sea level.[citation needed]

The Sinhagad (Lion's Fort) was strategically built to provide natural protection due to its very steep slopes. The walls and bastions were constructed only at key places. There are two gates to enter the fort, the Kalyan Darwaza and Pune Darwaza which are positioned at the south east and north-east ends respectively.[1] The fort was also strategically located at the centre of a string of other 

c:\python37\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
